In [1]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from ragas import EvaluationDataset

In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = InMemoryVectorStore(embedding=embeddings)

loader = WikipediaLoader(query="nashville history")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
vector_store.add_documents(split_documents)

['3a0adbde-4d22-499d-a1be-17e94f20827a',
 '8aa8f4f7-8f26-4c15-8daf-99a64193766f',
 '49ed4dae-ebe5-4767-bfbd-3f72da1471ff',
 'bc1e00a7-b63c-4f9b-b717-59117997afcb',
 '3f746736-89af-4bb2-ba8a-5aee71773e6d',
 '428fcf89-9634-4d41-8734-6b40f63eceaa',
 'df827b58-5e06-438a-bceb-a4e615f871de',
 'd12757e9-1904-44c8-b705-410afea6e26b',
 '9e12a178-cc8f-4125-9108-f06fd266b555',
 '66e71ce7-7366-4699-b9e8-fde787464ce1',
 '793212f9-6582-42f6-aa47-f2a0e8341a39',
 '3b95b6e0-207d-44c9-986e-0f3090d2cf9c',
 'e68491e4-bc98-4817-94b5-f55430ec8447',
 '717085d3-3fbe-4960-84b2-27b43732b6a4',
 '5f512a21-f13f-477b-9c74-fcd7524822d9',
 '25f2200e-915f-4e96-bae2-d0f28f02ae29',
 '4a691260-0dd2-4c13-a6b1-92b83064d5a0',
 '24251bb8-5734-4715-9038-e87d19f3dc3c',
 '11afb2b6-52b7-47a9-8215-eaf0d05f70ea',
 '42c311b9-cd97-4743-a00a-7c9e309c9d0f',
 'ed098d10-412e-4f01-a8e6-36437be34e48',
 '44520f01-a671-4f5d-b4c0-50aeed106e76',
 '900cf7c7-f3d4-4612-8c02-99125f1a58f0',
 'b8c27d9b-ec71-48ba-af46-59fab8b54c87',
 '1b9ef3a3-5dd5-

In [3]:
sample_queries = [
    "What is the capital of Tennessee?",
    "When was Nashville founded?",
    "What was the original name of Nashville?",
]

expected_responses = [
    "The capital of Tennessee is Nashville.",
    "Nashville was founded in 1779.",
    "The original name of Nashville was Fort Nashborough.",
]

In [4]:
from langchain_openai import ChatOpenAI

system_prompt = (
    "You are a helpful assistant. "
    "Answer questions based on the provided context."
    "If no context is provided or you cannot source the response from the provided context, "
    "then you politely refuse to answer the question."
    "Give direct responses to the user's question and do not give extraneous information."
)

llm = ChatOpenAI(model="gpt-4o-mini")

messages_with_system_prompt = [SystemMessage(content=system_prompt)]

In [5]:
dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_documents = vector_store.similarity_search(query)
    messages_with_system_prompt.extend(
        [
            HumanMessage(content=f"{query}"),
            HumanMessage(content=f"context: {relevant_documents}"),
        ]
    )
    response = llm.invoke(messages_with_system_prompt).content
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [
                document.page_content for document in relevant_documents
            ],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [6]:
evaluation_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What is the capital of Tennessee?,"[Nashville, often known as Music City, is the ...",The capital of Tennessee is Nashville.,The capital of Tennessee is Nashville.
1,When was Nashville founded?,[The following is a timeline of the history of...,Nashville was founded in 1779.,Nashville was founded in 1779.
2,What was the original name of Nashville?,[== History ==\nNashville was originally calle...,The original name of Nashville was Fort Nashbo...,The original name of Nashville was Fort Nashbo...


In [7]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import FactualCorrectness, Faithfulness, LLMContextRecall

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

result

Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

{'context_recall': 0.6667, 'faithfulness': 0.6667, 'factual_correctness': 1.0000}

In [8]:
df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness
0,What is the capital of Tennessee?,"[Nashville, often known as Music City, is the ...",The capital of Tennessee is Nashville.,The capital of Tennessee is Nashville.,1.0,1.0,1.0
1,When was Nashville founded?,[The following is a timeline of the history of...,Nashville was founded in 1779.,Nashville was founded in 1779.,1.0,1.0,1.0
2,What was the original name of Nashville?,[== History ==\nNashville was originally calle...,The original name of Nashville was Fort Nashbo...,The original name of Nashville was Fort Nashbo...,0.0,0.0,1.0


In [10]:
print(df["retrieved_contexts"][2])

['== History ==\nNashville was originally called New Nashville; under the latter name, it was laid out in 1830. The local post office was established as Nashville in 1831. On June 28, 2020, Nashville was the site of a successful attempt at the world record for most pogo stick jumps with no hands. Caleb Klein set the record with 13,015 consecutive jumps.', 'Named for Francis Nash, a general of the Continental Army during the American Revolutionary War, the city was founded in 1779 when this territory was still considered part of North Carolina. The city grew quickly due to its strategic location as a port on the Cumberland River and, in the 19th century, a railroad center. Nashville as part of Tennessee seceded during the American Civil War; in 1862 it was the first state capital in the Confederacy to be taken by Union forces. It was occupied', 'Nashville was founded about 1840. In 1856, Nashville was designated seat of the newly formed Berrien County. It was incorporated as a town in 1

In [11]:
vector_store = InMemoryVectorStore(embedding=embeddings)

loader = WikipediaLoader(query="nashville tennessee history")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
vector_store.add_documents(split_documents)

['3f97fded-acf2-4842-beba-77278e874a78',
 '1d7b517d-b271-438d-9dff-dedafe4fd6bb',
 '4f966351-0da8-4e79-b812-082e2a6209ee',
 '974536ea-00d4-4f9a-afd8-bce9e80c60ef',
 'e6edc546-923b-44a5-bec6-d4b4ef156fac',
 'e4f049a5-8ac8-4172-83c2-b7013e6e4ad7',
 '9d79b4d6-6719-46b8-bef0-3188ff8fccca',
 '46840556-b79c-414a-aa47-d5ddf98a6f89',
 '5be399ae-e9ed-49cd-b33b-87303f39c93a',
 'a5d52477-bc61-4fc5-b8da-8d2f3cbeadf5',
 'cf35222c-41d5-4a89-8394-21f8ba1be548',
 'cdf0b672-7088-41a2-80c6-e7ea05cfdaa4',
 '16f51ee5-a5ee-4ad6-9079-dab6de40813c',
 'a1801490-3a71-4a0e-9556-6e034c0903e0',
 'fb26fb15-9e9f-45fe-ae07-6a1c19028022',
 '636bb4c1-44d7-4b21-b406-a816fb878460',
 'b5aded72-4384-4aec-85ad-4760f0991b48',
 '7979b50c-3c18-46f8-8f38-c5f34968e674',
 '715094ac-70b4-4c6f-aa57-4752a264f8fe',
 '018f1ad9-2f09-4f9e-9be3-d6d028956fcc',
 '72fcc4ba-db47-4d8d-9c5a-fb80d9e830bc',
 'abaf3edb-b639-48e2-a35a-5e4475c1f561',
 '6b7d83e8-4055-4325-a313-d182aded5bbb',
 '19d95c8e-f233-4457-ac1b-ffdb1022f3f9',
 'e796d160-e9e9-

In [12]:
dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_documents = vector_store.similarity_search(query)
    messages_with_system_prompt.extend(
        [
            HumanMessage(content=f"{query}"),
            HumanMessage(content=f"context: {relevant_documents}"),
        ]
    )
    response = llm.invoke(messages_with_system_prompt).content
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [
                document.page_content for document in relevant_documents
            ],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [13]:
evaluation_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What is the capital of Tennessee?,[=== Background and planning ===\nWhen Tenness...,The capital of Tennessee is Nashville.,The capital of Tennessee is Nashville.
1,When was Nashville founded?,[The following is a timeline of the history of...,Nashville was founded in 1779.,Nashville was founded in 1779.
2,What was the original name of Nashville?,"[Named for Francis Nash, a general of the Cont...",The original name of Nashville was Fort Nashbo...,The original name of Nashville was Fort Nashbo...


In [14]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import FactualCorrectness, Faithfulness, LLMContextRecall

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

result

Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'faithfulness': 1.0000, 'factual_correctness': 1.0000}

In [15]:
df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness
0,What is the capital of Tennessee?,[=== Background and planning ===\nWhen Tenness...,The capital of Tennessee is Nashville.,The capital of Tennessee is Nashville.,1.0,1.0,1.0
1,When was Nashville founded?,[The following is a timeline of the history of...,Nashville was founded in 1779.,Nashville was founded in 1779.,1.0,1.0,1.0
2,What was the original name of Nashville?,"[Named for Francis Nash, a general of the Cont...",The original name of Nashville was Fort Nashbo...,The original name of Nashville was Fort Nashbo...,1.0,1.0,1.0
